In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install langchain==0.0.327
!pip install langchain-community==0.0.25
!pip install langchain-core==0.1.28
!pip install langchain-openai==0.0.8
!pip install faiss-cpu==1.7.4
!pip install openai==0.28.1
!pip install transformers==4.38.1
!pip install tiktoken==0.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: anyio
    Found existing installation: anyio 4.2.0
    Uninstalling anyio-4.2.0:
      Successfully uninstalled anyio-4.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.1.6 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 16.0 MB/s eta 0:00:00
  

In [3]:
# GenAICentral-cust-fn-embed-vdb-12-04

In [4]:
import os
import re
import sys
import json
import time
import openai
import pickle
import datetime
import numpy as np
import pandas as pd
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain.docstore.document import Document
from langchain.embeddings.openai import OpenAIEmbeddings

In [5]:
### Configuration- for azure
root_path = '/content/drive/MyDrive/Data/'
openai.api_type = "azure"
openai.api_base = "https://asdf.openai.azure.com/"
openai.api_key =  ""
openai.api_version = "2023-10-15-preview"
model_name = "text-embedding-ada-002"
engine_name = "TEA"
chunk_size = 512

In [6]:
def read_xcel_data(file_path):

    documents = []
    num_documents = []

    try:
        df = pd.read_excel(file_path)
        data = df.astype(str)
        cols = ['id', 'name', 'description','rootCause', 'solution']

        data = data[cols]
        json_lines = [json.dumps(row.to_dict()) for index , row in data.iterrows()]
        documents = [Document(page_content = json_line) for json_line in json_lines]
        print(f"The json conversion is done.")

        num_data = data['issueNum']
        num_json_lines = [json.dumps({'issueNum':row}) for index , row in num_data.items()]
        num_documents = [Document(page_content = json_line) for json_line in num_json_lines]
        print(f"The json conversion is done for Issue Number Docs.")

    except Exception:
        print("Error in parsing the excel file")
        raise Exception

    return documents, num_documents

In [7]:
 def create_vectordb(documents,embeddings,path):
    try:
        if len(documents) <= 500:
            print(f"Small file, using direct call {len(documents)}")
            db = FAISS.from_documents(documents, embeddings)

        else:
            print(f"Large file recieved {len(documents)}.")
            batch_size = 500
            print(f"Batch size is {batch_size}")
            db = FAISS.from_documents(documents[:batch_size], embeddings)
            index = batch_size

            while index < len(documents) :
                db_temp = FAISS.from_documents(documents[index:index+batch_size], embeddings)
                db.merge_from(db_temp)
                index+=batch_size
                print(f"Progess of DB {index}")

    except openai.error.RateLimitError :
            print(f"getting timeout error so running long process for{len(documents)} documents.")
            print(documents[0])
            db = FAISS.from_documents(documents[0:1], embeddings)
            index = 1
            while index < len(documents) :
                db_temp = FAISS.from_documents(documents[index:index+1], embeddings)
                db.merge_from(db_temp)
                index+=1
                time.sleep(0.12)
    db.save_local(path)

In [8]:
def get_azure_openAI_embeddings():

    embeddings = ""
    try:
        embeddings =OpenAIEmbeddings(
                                openai_api_key = openai.api_key,
                                model = model_name,
                                engine=engine_name,
                                deployment=engine_name,
                                chunk_size=chunk_size)

    except Exception:
        print("Error in getting openAI embeddings")
        raise Exception

    return embeddings

In [9]:
def get_faiss_index(path):

    with open(path, 'rb') as f:
        data = pickle.load(f)

    idx = list(data[1].values())
    return idx

In [10]:
def embedd(file_path,vdb_name):

    db = None
    num_db = None
    num_vdb_name = 'num_' + vdb_name
    try:
        print("file_path %s",file_path)
        print("Vectordb name %s",vdb_name)
        print("Issue Num Vectordb name %s",vdb_name + 'num')
        print("Model Name %s", model_name)
        print("chunk size %s", chunk_size)
        file_extension = file_path.split(".")[-1]
        print("file extension %s",file_extension)

        print("\n\nStarting Embedding for files")
        #get embeddings
        embeddings = get_azure_openAI_embeddings()

        #for excel files
        if file_extension == "xlsx" :
            print("xlsx file given as input to parser")
            documents, num_documents = read_xcel_data(file_path)

        elif file_extension == "csv" :
            print("csv file given as input to parser")
            documents, num_documents = read_csv_data(file_path)

        else:
            print("File type is not supported.")
            print("file extention is not xlsx please check the file type and proceed")

        #TODO pass json data to below line
        print("Creating Vector db for complete docs and vector_db path is ",root_path + vdb_name)
        # documents, num_documents = read_xcel_data(file_path)
        create_vectordb(documents,embeddings,root_path + vdb_name)
        time.sleep(60)
        print("Creating Vector db for Issue Number docs and vector_db path is ",root_path + num_vdb_name)
        create_vectordb(num_documents,embeddings,root_path + num_vdb_name)

        # Document Swapping
        try:
            db_idx = get_faiss_index(root_path + vdb_name + '/index.pkl')
            num_db_idx = get_faiss_index(root_path + num_vdb_name + '/index.pkl')

            db = FAISS.load_local(root_path + vdb_name,embeddings)
            num_db = FAISS.load_local(root_path + num_vdb_name,embeddings)

            for i in range(len(db_idx)):
                replace_docs = db.docstore._dict[db_idx[i]]
                num_db.docstore._dict[num_db_idx[i]] = replace_docs

            num_db.save_local(root_path + num_vdb_name)

        except KeyError:
            print("Error while swapping documents")


    except Exception:
        print("Error in generating embeddings")
        raise Exception

In [11]:
# embedd('/content/drive/MyDrive/Data/data_len/qqdts-data.csv','qqdts_vdb')

# output--
# file_path %s /content/drive/MyDrive/Data/data_len/qqdts-data.csv
# Vectordb name %s qqdts_vdb
# Issue Num Vectordb name %s qqdts_vdbnum
# Model Name %s text-embedding-ada-002
# chunk size %s 512
# file extension %s csv


# Starting Embedding for files
# csv file given as input to parser
# /usr/local/lib/python3.10/dist-packages/langchain/embeddings/openai.py:217: UserWarning: WARNING! engine is not default parameter.
#                     engine was transferred to model_kwargs.
#                     Please confirm that engine is what you intended.
#   warnings.warn(
# The json conversion is done.
# The json conversion is done for Issue Number Docs.
# Creating Vector db for complete docs and vector_db path is  /content/drive/MyDrive/Data/qqdts_vdb
# Small file, using direct call 491
# Creating Vector db for Issue Number docs and vector_db path is  /content/drive/MyDrive/Data/num_qqdts_vdb
# Small file, using direct call 491

In [12]:
def azure_llm(temperature, max_tokens, prompt):

    response = openai.ChatCompletion.create(
              engine = engine,
              # Here prompt is message text
              messages = prompt,
              temperature = temperature,
              max_tokens = max_tokens,
              top_p=0.2,
              frequency_penalty = 0,
              presence_penalty = 0)


    # text = response['choices'][0]['message']['content'].strip()
    text = response.choices[0].message.content
    return text

In [13]:
# db = FAISS.load_local('/content/drive/MyDrive/Data/QQDTS-cases-data-240424/new_qdts_data_fixed0205',embeddings)

prompts = """
Answer the question from the given context.
Keep answer to be precise and informative.
Context is a customer ticket data created for the products having issue.

question: {question}

context : {context}
"""

def llm_answer(query):
    temperature = 0.1
    max_tokens = 200

    retriever = db.as_retriever(search_kwargs = {"k": 5})
    documents = retriever.get_relevant_documents(query)

    context = ""
    for page in documents:
        context+=page.page_content

    prompt = ChatPromptTemplate.from_template(prompts).format_messages(
    question = query,
    context = context
    )
    message_text=[{"role": "system", "content": prompt[0].content}]

    resp = azure_llm(temperature, max_tokens, message_text)
    return resp, message_text